In [1]:
import os
import numpy as np
import tensorflow_datasets as tfds
import tensorflow as tf
import matplotlib.pyplot as plt
import keras
from keras import layers
from tensorflow import data as tf_data

In [ ]:
cropName = "Mango"

In [2]:
image_size = 224
batch_size = 32
train_path = f"./PlantDisease/{cropName}/All/"
train_ds, val_ds = keras.utils.image_dataset_from_directory( 
    directory=train_path,
    subset= 'both', 
    seed=1337,
    validation_split=0.2,
    image_size = (image_size, image_size),
    batch_size = batch_size,
    label_mode ='categorical'
)

train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

Found 39703 files belonging to 5 classes.
Using 31763 files for training.
Using 7940 files for validation.


In [3]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"), 
    layers.RandomRotation (0.1),
]
def data_augmentation(images):
    for layer in data_augmentation_layers: 
        images = layer(images)
    return images

In [4]:
train_ds = train_ds.map(
    lambda img, label: (data_augmentation (img), label), 
    num_parallel_calls = tf_data.AUTOTUNE,
)

train_ds = train_ds.prefetch(tf_data.AUTOTUNE)
val_ds = val_ds.prefetch(tf_data.AUTOTUNE)

In [5]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense 
from tensorflow.keras.applications import MobileNet

num_classes=len(os.listdir("All/"))

inputs = layers.Input(shape=(image_size, image_size, 3))
model = MobileNet(include_top=False, input_tensor=inputs, weights="imagenet") 
model.trainable = False
x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
x = layers.BatchNormalization()(x)

top_dropout_rate = 0.2
x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
outputs = layers.Dense(num_classes, activation="softmax", name="pred") (x)
model = keras.Model(inputs, outputs, name="MobileNet")
optimizer = keras.optimizers.Adam(learning_rate=1e-2) 
model.compile(
    optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
)
model.summary(show_trainable=True)

C:\Users\DELL\AppData\Local\Temp\ipykernel_16764\2034215482.py:8: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  model = MobileNet(include_top=False, input_tensor=inputs, weights="imagenet")


Model: "MobileNet"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ input_layer (InputLayer)    │ (None, 224, 224, 3)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv1 (Conv2D)              │ (None, 112, 112, 32)  │        864 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv1_bn                    │ (None, 112, 112, 32)  │        128 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv1_relu (ReLU)           │ (None, 112, 112, 32)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_1 (DepthwiseConv2D) │ (None, 112, 112, 32)  │        288 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_1_bn                │ (None, 112, 112, 32)  │        128 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_1_relu (ReLU)       │ (None, 112, 112, 32)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_1 (Conv2D)          │ (None, 112, 112, 64)  │      2,048 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_1_bn                │ (None, 112, 112, 64)  │        256 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_1_relu (ReLU)       │ (None, 112, 112, 64)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pad_2 (ZeroPadding2D)  │ (None, 113, 113, 64)  │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_2 (DepthwiseConv2D) │ (None, 56, 56, 64)    │        576 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_2_bn                │ (None, 56, 56, 64)    │        256 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_2_relu (ReLU)       │ (None, 56, 56, 64)    │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_2 (Conv2D)          │ (None, 56, 56, 128)   │      8,192 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_2_bn                │ (None, 56, 56, 128)   │        512 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_2_relu (ReLU)       │ (None, 56, 56, 128)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_3 (DepthwiseConv2D) │ (None, 56, 56, 128)   │      1,152 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_3_bn                │ (None, 56, 56, 128)   │        512 │   N   │
│ (BatchNormalization)        │                       │            │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_dw_3_relu (ReLU)       │ (None, 56, 56, 128)   │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_3 (Conv2D)          │ (None, 56, 56, 128)   │     16,384 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ conv_pw_3_bn                │ (None, 56, 56, 128)   │        512 │   N 

 Total params: 3,238,085 (12.35 MB)

 Trainable params: 7,173 (28.02 KB)

 Non-trainable params: 3,230,912 (12.32 MB)

In [6]:
epochs = 5
hist = model.fit(train_ds, epochs = epochs , validation_data = val_ds)

Epoch 1/5
993/993 ━━━━━━━━━━━━━━━━━━━━ 311s 311ms/step - accuracy: 0.8934 - loss: 0.3799 - val_accuracy: 0.9491 - val_loss: 0.2055
Epoch 2/5
993/993 ━━━━━━━━━━━━━━━━━━━━ 279s 280ms/step - accuracy: 0.9356 - loss: 0.2872 - val_accuracy: 0.9691 - val_loss: 0.1588
Epoch 3/5
993/993 ━━━━━━━━━━━━━━━━━━━━ 261s 263ms/step - accuracy: 0.9419 - loss: 0.2561 - val_accuracy: 0.9767 - val_loss: 0.0854
Epoch 4/5
993/993 ━━━━━━━━━━━━━━━━━━━━ 264s 266ms/step - accuracy: 0.9436 - loss: 0.2520 - val_accuracy: 0.9727 - val_loss: 0.0868
Epoch 5/5
993/993 ━━━━━━━━━━━━━━━━━━━━ 272s 274ms/step - accuracy: 0.9409 - loss: 0.2542 - val_accuracy: 0.9758 - val_loss: 0.0939


In [7]:
model.save(f"User/CropDisease/DiseaseModels/{cropName}.h5")